In [1]:
# Plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Dash&Flask
from jupyter_dash import JupyterDash
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
from flask_caching import Cache
# Onnx
import onnx
import onnxruntime
from onnx_tf.backend import prepare
# Custom libraries
import layouts
from layouts import images_page, text_page
import utilities
import callbacks
from callbacks import app
# Others
import os
import base64
import numpy as np
import dianna
import warnings
warnings.filterwarnings('ignore') # disable warnings relateds to versions of tf

In [ ]:
# debug
from importlib import reload
reload(layouts)
reload(utilities)

In [3]:
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/apps/text':
         return text_page
    else:
         return images_page # home page

app.run_server(mode='external', port=8050)

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.
Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


Dash app running on http://127.0.0.1:8050/


Explaining: 100%|██████████| 50/50 [00:00<00:00, 62.72it/s]
2022-04-05 13:57:15.715959: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-05 13:57:15.716127: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-04-05 13:57:18.010491: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-05 13:57:18.010545: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Explaining: 100%|██████████| 50/50 [00:00<00:00, 63.67it/s]
2022-04-05 13:57:33.953613: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
